In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import array
# import string
# from string import punctuation
# from itertools import chain

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, f1_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_selection import SelectPercentile, chi2, f_regression, f_classif
from sklearn.utils import shuffle

! pip install imbalanced-learn
from imblearn.over_sampling import SMOTE
# from sklearn import svm

# nltk.download('stopwords')
# from nltk.stem import WordNetLemmatizer
# from nltk import ngrams

import re
import string

! pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')


! pip install wordcloud
from wordcloud import WordCloud, STOPWORDS

! pip install tensorflow

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense, Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
import io
import json

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\kaila\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kaila\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kaila\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
analysis_set = pd.read_csv("csv's/AnalysisDataset.csv")

In [30]:
reviews = analysis_set["cleanedText"]
reviews = reviews.apply(lambda x: re.sub(r"\s+", " ", str(x)).strip() if isinstance(x, (str, float, int)) else x)

In [31]:
unseen_reviews = []
for review in reviews:
    unseen_reviews.append(review)

In [32]:
unseen_reviews

['really like ear swab first come large handy box easy store last long time second organic good environment third strong fall apart easily highly recommend used year',
 'honest rarely used overnight cream typically skin care routine morning night wash apply hyalauronic serum kind moistutrizer saw iryasa cream wanted try easy apply immediately feel moisture starting work apply minute actually go sleep allow time absorbed putting face pillow thicker type cream need time skin morning feel better ever soft supple definitely feel difference using week highly recommend cream woman men',
 'purchased see could set mom hair fine thin short hair think could fun fluffy set mom super old school hoping little foam styling product',
 'great multi pack band great price wash well laundry lose elasticity',
 'easy use',
 'good way use face oil normally skip facial oil like way feel face one mixed cream leave face feeling greasy oily applying pleasant scent sticky though leave behind little tackiness dis

In [33]:
with open('embedded.json') as f:
    data = json.load(f)
    loaded_tokenizer = tokenizer_from_json(data)

In [34]:
unseen_tokenized = loaded_tokenizer.texts_to_sequences(unseen_reviews)
unseen_padded = pad_sequences(unseen_tokenized, padding='post', maxlen=100)

In [50]:
from keras.models import load_model

model_path = 'lstm_model.keras'
pretrained_lstm_model = load_model(model_path)
pretrained_lstm_model.summary()
preds = pd.Series()
preds = pretrained_lstm_model.predict(unseen_padded)
# unseen_sentiments

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 100)       │     6,029,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,381,833 (24.34 MB)

 Trainable params: 117,377 (458.50 KB)

 Non-trainable params: 6,029,700 (23.00 MB)

 Optimizer params: 234,756 (917.02 KB)

1183/1183 ━━━━━━━━━━━━━━━━━━━━ 16s 14ms/step


In [51]:
analysis_set["predictedRating"] = np.round(preds * 5, 1)

df_prediction = pd.DataFrame(analysis_set['predictedRating'], columns = ['predictedRating'])
# df_userName = pd.DataFrame(testing_set["userName"], columns=["userName"])
# df_itemName = pd.DataFrame(df["product_name"], columns=["product_name"])
# df_category = pd.DataFrame(df["category"], columns=["category"])
# df_price = pd.DataFrame(df["discounted_price"], columns=["discounted_price"])
df_rating = pd.DataFrame(analysis_set["rating"], columns=["rating"])

dfx = pd.concat([
                # df_itemName, 
                #  df_category, df_price, 
                df_rating, df_prediction], axis=1)

dfx.head(2)

,rating,predictedRating
0,5.0,5.0
1,5.0,5.0


In [37]:
dfx.to_csv("Predictions.csv")

## Custom popularity metric
Popularity Score (PS)
PS= 𝛼 × Number of Reviews + 𝛽 × Average Rating+ 𝛾 × Total Votes

Where:

Number of Reviews = total reviews per item per month (or week).

Average Rating = mean of ratings per item per month.

Total Votes = sum of votes (helpfulness) on the reviews per item per month.

α, β, γ = tunable weights based on what you want to emphasize more.

In [ ]:
df.columns